In [2]:
from tkinter import *
import sounddevice as sd
from scipy.io.wavfile import write
import os, sys, subprocess
import numpy as np
import json
import librosa
import noisereduce as nr
import tkinter as tk

root = Tk()

root.geometry("600x600")
root.title("Recorder")

def open_file(filename):
    if sys.platform == "win32":
        os.startfile(filename)
    else:
        opener ="open" if sys.platform == "darwin" else "xdg-open"
        subprocess.call([opener, filename])
        

def record():
    global data
    fs = 44100  
    seconds = .75
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    sd.wait()
    data =  myrecording

def play():
    global update,fr
    write('/home/incentive/Videos/Group Project/Digit Recognition/Application/Data/' +  'temp'+'.wav', 44100, data)
    audio_data, sampling_rate = librosa.load('/home/incentive/Videos/Group Project/Digit Recognition/Application/Data/'
                                             +'temp.wav')
    noisy_part = audio_data[10000:15000]  
    reduced_noise = nr.reduce_noise(audio_clip=audio_data, noise_clip=noisy_part, verbose=False)
    update = reduced_noise
    fr=sampling_rate
    sd.play(reduced_noise,sampling_rate)
    sd.wait(6)
    
def save():
    label = clicked_level.get()
    user = clicked_user.get()
    label_dict[user][label] += 1
    write('/home/incentive/Videos/Group Project/Digit Recognition/Application/Data/' +
          label+'_'+user+'_'+str(label_dict[user][label])+'.wav', fr,update)
    json.dump(label_dict, open("label.txt",'w'))
    myLabel = Label(root, text='File save as '+label+'_'+user+'_'+str(label_dict[user][label])+'.wav').pack()  
    
    
tk.Label(root, 
		 text="Record Digit Data",
		 fg = "Red",
		 font = "arial 30 bold italic").pack(pady=20)

option_level = ["0","1","2","3","4","5","6","7","8","9",]
user_option = ["atik","polash","sorif","adeeb","dibyendu","abir","mazhar","tuli","tasmina","rajosree","saiful",]


label_dict = json.load(open("label.txt"))


clicked_level = StringVar()
clicked_level.set(option_level[0])
clicked_user = StringVar()
clicked_user.set(user_option[0])

drop = OptionMenu(root, clicked_level, *option_level)
drop.pack()

drop_user = OptionMenu(root, clicked_user, *user_option)
drop_user.pack()


record_button = Button(root, text="Start",bg="blue", command=record,font = "Helvetica 20 bold italic").pack(pady=10)

play_button = Button(root, text="Play",bg="red", command=play,font = "Helvetica 20 bold italic").pack(pady=10)
save_button = Button(root, text="Save",bg="green", command=save,font = "Helvetica 20 bold italic").pack(pady=10)


root.mainloop() 